# Transfer Learning with TensorFlow Part 3: Scaling up (Food Vision min)

We've seen the power of transfer learning feature extraction and fine tuning, now it's time to scale up to all of the classes in Food101 (101 total classes of food).

Our goal is beat the original Food101 paper with 10% of the training data (leveraging the power of deep learning)

In [1]:
# Check to see of we're using a GPU
!nvidia-smi

Fri Sep 29 12:38:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.56.06    Driver Version: 522.30       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   50C    P8    23W / 170W |  10395MiB / 12288MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions

In previous notebooks, we've created a series of helper functions to do different tasks, let's download them.

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-09-29 12:41:10--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.002s  

2023-09-29 12:41:10 (5.76 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [3]:
# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## 101 Food Classes: working with less data

Our goal is to beat the original Food101 paper with 10% of the training data, sot let's download it.

The data we're downloading comes from the original Food101 dataset but has been preprocessed using the imag_data_modification notebook.

In [4]:
assets_dir = "assets"

In [6]:
import os
# !wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data(os.path.join(assets_dir, "101_food_classes_10_percent.zip"))

In [10]:
train_dir = os.path.join(assets_dir, "101_food_classes_10_percent", "train")
test_dir = os.path.join(assets_dir, "101_food_classes_10_percent", "test")

In [11]:
# How many iamges/classes are there?
walk_through_dir(os.path.join(assets_dir, "101_food_classes_10_percent"))

There are 2 directories and 0 images in 'assets/101_food_classes_10_percent'.
There are 101 directories and 0 images in 'assets/101_food_classes_10_percent/test'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/apple_pie'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/baby_back_ribs'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/baklava'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/beef_carpaccio'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/beef_tartare'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/beet_salad'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/beignets'.
There are 0 directories and 250 images in 'assets/101_food_classes_10_percent/test/bibimbap'.
There are 0 directories and 250 images in 'assets/101_food_classes_

There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/cannoli'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/caprese_salad'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/carrot_cake'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/ceviche'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/cheesecake'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/cheese_plate'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/chicken_quesadilla'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in 'assets/101_food_classes_10_percent/train/chocolate_cake'.
There are 0 directorie

In [12]:
# Steup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                                       label_mode="categorical",
                                                                       image_size=IMG_SIZE)

test_data = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                       label_mode="categorical",
                                                       image_size=IMG_SIZE,
                                                       shuffle=False) # don't shuffle test data fo prediction analysis

Found 7575 files belonging to 101 classes.


2023-09-29 13:30:56.540196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-29 13:30:56.547013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-29 13:30:56.547365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-29 13:30:56.548833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-29 13:30:56.549148: I tensorflow/compile

Found 25250 files belonging to 101 classes.
